In [3]:
#pip install pytesseract

In [4]:
import pytesseract
from PIL import Image
import os

# Ruta al dataset
dataset_path = 'dataset/eu'

# Lista para almacenar los textos extraídos
extracted_texts = []

# Iterar sobre las imágenes en la carpeta
for filename in os.listdir(dataset_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(dataset_path, filename)
        image = Image.open(image_path)
        
        # Extraer texto usando Tesseract
        text = pytesseract.image_to_string(image)
        extracted_texts.append((filename, text))

# Mostrar los textos extraídos
for filename, text in extracted_texts:
    print(f"Texto extraído de {filename}:\n{text}\n")

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.